# Load Libraries

In [2]:
import pandas as pd
import numpy as np

# Load Data

In [ ]:
filepath = "/Users/Kapil/PycharmProjects/stock-price"
train = pd.read_csv(filepath + "/data/train.csv")
test = pd.read_csv(filepath + "/data/test.csv")

In [13]:
SEED = 13
np.random.seed(SEED)

In [14]:
train.shape

(2026, 6)

In [11]:
train.head()

,Date,Open,High,Low,Close,Up
0,2004-01-02,50.85,50.91,50.62,50.72,1
1,2004-01-05,50.75,50.84,50.61,50.79,1
2,2004-01-06,50.87,51.26,50.86,51.15,1
3,2004-01-07,51.45,51.54,51.29,51.51,0
4,2004-01-08,51.15,51.33,51.02,51.28,0


In [15]:
Y_train = train['Up']
X_train = train.drop(['Up'], axis = 1)

X_test = test